In [1]:
import pandas as pd
# importing/instantiating all the csv's in case I want to use any of the other data as well
# main DF
df = pd.read_csv('C:/users/Stewa/Documents/Downloads/steam-store-games/steam.csv')
df_clean = df.dropna()

In [28]:
# make my target and features
target = 'price'
features = ['english', 'developer', 'publisher', 'platforms', 'required_age',
            'categories', 'genres', 'steamspy_tags', 'achievements', 'positive_ratings',
            'negative_ratings',	'average_playtime', 'median_playtime', 'owners']

# break up my dataset
# first, I need to cast release_date to dt format
df_clean['release_date'] = pd.to_datetime(df_clean['release_date'], infer_datetime_format=True)
train = df_clean.loc[(df_clean['release_date'] <= pd.datetime(2014,12,31)) 
                     & (df_clean['release_date'] >= pd.datetime(1997,1,1))]

val = df_clean.loc[(df_clean['release_date'] >= pd.datetime(2015,1,1))
                   & (df_clean['release_date'] <= pd.datetime(2016,12,31))] 

test = df_clean.loc[(df_clean['release_date'] >= pd.datetime(2017,1,1))
                   & (df_clean['release_date'] <= pd.datetime(2019,12,31))]

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

C:\Users\stewa\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [29]:
import category_encoders as ce
o_encoder = ce.OrdinalEncoder()
X_train_enc = o_encoder.fit_transform(X_train)
X_test_enc = o_encoder.transform(X_test)
X_val_enc = o_encoder.transform(X_val)
X_train_enc

,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners
0,1,1,1,1,0,1,1,1,0.0,124534.0,3339.0,17612.0,317.0,1
1,1,1,1,1,0,1,1,1,0.0,3318.0,633.0,277.0,62.0,2
2,1,1,1,1,0,2,1,2,0.0,3416.0,398.0,187.0,34.0,2
3,1,1,1,1,0,1,1,1,0.0,1273.0,267.0,258.0,184.0,2
4,1,2,1,1,0,3,1,3,0.0,5250.0,288.0,624.0,415.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,1,2027,1347,3,0,740,42,1068,12.0,102.0,97.0,55.0,55.0,7
4409,1,2028,210,2,0,83,262,1881,17.0,510.0,107.0,14.0,14.0,12
4422,1,1630,1078,1,0,654,63,1882,72.0,1719.0,50.0,0.0,0.0,10
4427,1,1630,1078,1,0,741,63,1883,37.0,108.0,24.0,0.0,0.0,13


In [30]:
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from xgboost import XGBRegressor

# Im going to make a few pipelines to see what works best

#first pipeline is gradient boosting regressor
xgb_pipe = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(), 
    XGBRegressor(n_estimators=100, objective='reg:squarederror', n_jobs=-1)
)

xgb_pipe.fit(X_train_enc, y_train)
y_pred_xgb = xgb_pipe.predict(X_val_enc)
print('Gradient Boosting R^2:', r2_score(y_val, y_pred_xgb))
# thats not a very good score so far lest try random forest next

C:\Users\stewa\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\stewa\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Gradient Boosting R^2: 0.14331589266310174


In [31]:
# build the randomforest model with OEncoding and imputer
rfr_pipe = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestRegressor(n_estimators=200, n_jobs=-1)
)

# fit and print out r2 score
rfr_pipe.fit(X_train, y_train)
y_pred_rfr = rfr_pipe.predict(X_val)
print('Random forest regression R^2:', r2_score(y_val, y_pred_rfr))
# Wow thats not good at all, lets try to work on the gradient boosting then

Random forest regression R^2: 0.005004579178334856


In [32]:
transformers = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median')
)
X_train_transformed = transformers.fit_transform(X_train)
X_val_transformed = transformers.transform(X_val)



model = XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_transformed, y_train)

C:\Users\stewa\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\stewa\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[12:39:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [33]:
# lets check out feature importances
import eli5
from eli5.sklearn import PermutationImportance

# Calculate permutation importances
permuter = PermutationImportance(
    model, 
    scoring='r2', 
    n_iter=5, 
    random_state=42
)

permuter.fit(X_val_transformed, y_val)

PermutationImportance(cv='prefit',
                      estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                             colsample_bylevel=1,
                                             colsample_bynode=1,
                                             colsample_bytree=1, gamma=0,
                                             importance_type='gain',
                                             learning_rate=0.1,
                                             max_delta_step=0, max_depth=3,
                                             min_child_weight=1, missing=None,
                                             n_estimators=100, n_jobs=-1,
                                             nthread=None,
                                             objective='reg:linear',
                                             random_state=42, reg_alpha=0,
                                             reg_lambda=1, scale_pos_weight=1,
                                     

In [34]:
feature_names = X_val.columns.tolist()

eli5.show_weights(
    permuter, 
    top=None, # show permutation importances for all features
    feature_names=feature_names # must be a list
)
#hmmm the font is white and impossible to read, will have to look into that

Weight,Feature
0.1382 ± 0.0296,median_playtime
0.0760 ± 0.0050,negative_ratings
0.0730 ± 0.0213,genres
0.0544 ± 0.0223,publisher
0.0280 ± 0.0163,positive_ratings
0.0236 ± 0.0083,developer
0.0224 ± 0.0147,achievements
0.0158 ± 0.0047,steamspy_tags
0.0062 ± 0.0055,platforms
0.0050 ± 0.0009,english
